In [234]:
import numpy as np
import pandas as pd
from random import random
from math import exp

In [235]:
data = pd.read_csv('data/seeds_dataset.txt',delim_whitespace=True,header=None)

In [236]:
data.head(10)

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
5,14.38,14.21,0.8951,5.386,3.312,2.462,4.956,1
6,14.69,14.49,0.8799,5.563,3.259,3.586,5.219,1
7,14.11,14.10,0.8911,5.420,3.302,2.700,5.000,1
8,16.63,15.46,0.8747,6.053,3.465,2.040,5.877,1
9,16.44,15.25,0.8880,5.884,3.505,1.969,5.533,1


In [237]:
features = list(data)[:-1]
target = 7

In [238]:
for feature in features:
    curr_max = max(data[feature])
    curr_min = min(data[feature])
    data[feature] = data[feature].apply(lambda x : x/(curr_max-curr_min))
data['output'] = data[target].apply(lambda x: [1 if i==x-1 else 0 for i in range(3)])

In [239]:
data.tail(5)

,0,1,2,3,4,5,6,7,output
205,1.151086,2.727273,7.970054,2.892455,2.124733,0.472116,2.397834,3,"[0, 0, 1]"
206,1.060434,2.661157,7.723230,2.894144,1.992160,0.562353,2.463319,3,"[0, 0, 1]"
207,1.246459,2.822314,8.060799,2.948198,2.303635,1.081148,2.489414,3,"[0, 0, 1]"
208,1.118036,2.729339,7.732305,2.913851,2.021383,0.467826,2.483506,3,"[0, 0, 1]"
209,1.161473,2.756198,7.880218,2.952140,2.119743,0.732944,2.492861,3,"[0, 0, 1]"


# Normalized the data set

### Buliding a network

In [240]:
def create_network(n_inputs,n_hidden,n_outputs):
    network = []
    # initializing random weights for hidden and output layer
    hidden = [{'weights':[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden)
    output = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output)
    return network
    

In [241]:
from random import seed
seed(1)
test_network = create_network(2,1,2)
for layer in network:
    print layer

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


# Forward propagation

### Activating neurons

In [242]:
def activate_neuron(weights,inputs):
    activation = weights[-1] #initializing with bias weight
    for i in range(len(weights)-1):
        activation += weights[i]*inputs[i]
    return activation

### Transferring neuron data

In [243]:
def transfer_neuron_data(activation):
    return 1/(1+exp(-1*activation))

### Propagating information forward

In [244]:
def forward_propagation(network,inputs):
    inputs = list(inputs)
    for layer in network:
        next_inputs = []
        for neuron in layer:
            
            activation = activate_neuron(neuron['weights'],inputs)
            neuron['output'] = transfer_neuron_data(activation)
            next_inputs.append(neuron['output'])
        inputs = next_inputs
    return inputs

# Testing forward propagation with dummy network

In [246]:
row = [1,0,None]
output = forward_propagation(test_network,row)
print output

[0.6629970129852887, 0.7253160725279748]


# Backward propagation

### Sending derivative backward

In [247]:
def send_derivative(output):
    return output*(1.0-output)

### propagating error backwards

In [251]:
def backward_propagate_error(network,output):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network)-1: # All networks except the last one
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i+1]:
                    error += (neuron['weights'][j]*neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)): # error for the last layer
                neuron = layer[j]
                error = output[j] - neuron['output']
                errors.append(error)
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j]*send_derivative(neuron['output'])

## Testing backward propagation

In [252]:
expected = [0,1]
backward_propagate_error(test_network,expected)
for layer in test_network:
    print layer

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.002711797799238243}]
[{'output': 0.6629970129852887, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14813473120687762}, {'output': 0.7253160725279748, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.05472601157879688}]


# Training Network now


### updating weights

In [189]:
def update_weights(network,inputs,l_rate):#l_rate is the learning rate
    for i in range(len(network)):
        inputs = inputs[:-1] # No need to update dummy neurons
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1] += l_rate*neuron['delta']
    
    

## Iterations to improve error

In [253]:
def train_network(network,data,l_rate,n_iter,n_outputs):
    for n in range(n_iter):
        sum_error = 0
        for i in range(len(data)):
            row = list(data[features].iloc[i])
            prediction = forward_propagation(network,row)
            expectation = data['output'].iloc[i]
            sum_error += sum([(expectation[j] - prediction[j])**2 for j in range(len(prediction))])
            backward_propagate_error(network,expectation)
            update_weights(network,row,l_rate)
        print "Iteration : ",n,"l_rate : ",l_rate," Error : ",sum_error
            

In [259]:
test_network = create_network(len(features),10,3)
train_network(test_network,data,0.5,100,3)

Iteration :  0 l_rate :  0.5  Error :  247.956691317
Iteration :  1 l_rate :  0.5  Error :  169.190797365
Iteration :  2 l_rate :  0.5  Error :  169.87459132
Iteration :  3 l_rate :  0.5  Error :  169.665486878
Iteration :  4 l_rate :  0.5  Error :  111.059703054
Iteration :  5 l_rate :  0.5  Error :  45.1978069118
Iteration :  6 l_rate :  0.5  Error :  45.0918188672
Iteration :  7 l_rate :  0.5  Error :  45.091666875
Iteration :  8 l_rate :  0.5  Error :  45.091665085
Iteration :  9 l_rate :  0.5  Error :  45.0916635045
Iteration :  10 l_rate :  0.5  Error :  45.0916619208
Iteration :  11 l_rate :  0.5  Error :  45.0916603333
Iteration :  12 l_rate :  0.5  Error :  45.0916587421
Iteration :  13 l_rate :  0.5  Error :  45.091657147
Iteration :  14 l_rate :  0.5  Error :  45.0916555481
Iteration :  15 l_rate :  0.5  Error :  45.0916539452
Iteration :  16 l_rate :  0.5  Error :  45.0916523382
Iteration :  17 l_rate :  0.5  Error :  45.0916507272
Iteration :  18 l_rate :  0.5  Error :  45

### Making predictions

In [263]:
def predict(network,inputs):
    output = forward_propagation(network,inputs)
    return output

In [273]:
print forward_propagation(test_network,data[features].iloc[11])

[0.02736703749828073, 0.04046183719327358, 0.9586015817064812]


In [262]:
data.head(100)

,0,1,2,3,4,5,6,7,output,prediction
0,1.440982,3.066116,7.903811,3.244932,2.360656,0.288783,2.570162,1,"[1, 0, 0]",3
1,1.405099,3.010331,7.995463,3.127252,2.375624,0.132364,2.440177,1,"[1, 0, 0]",3
2,1.349386,2.911157,8.212341,2.979167,2.378475,0.350934,2.375677,1,"[1, 0, 0]",3
3,1.306893,2.880165,8.126134,2.997748,2.408411,0.293724,2.365830,1,"[1, 0, 0]",3
4,1.524079,3.097107,8.197822,3.185811,2.538845,0.176182,2.548006,1,"[1, 0, 0]",3
5,1.357885,2.935950,8.122505,3.032658,2.360656,0.320119,2.440177,1,"[1, 0, 0]",3
6,1.387158,2.993802,7.984574,3.132320,2.322880,0.466265,2.569670,1,"[1, 0, 0]",3
7,1.332389,2.913223,8.086207,3.051802,2.353528,0.351064,2.461841,1,"[1, 0, 0]",3
8,1.570349,3.194215,7.937387,3.408221,2.469708,0.265249,2.893648,1,"[1, 0, 0]",3
9,1.552408,3.150826,8.058076,3.313063,2.498218,0.256017,2.724274,1,"[1, 0, 0]",3


In [200]:
data.head(100)

,0,1,2,3,4,5,6,7,output,prediction
0,1.440982,3.066116,7.903811,3.244932,2.360656,0.288783,2.570162,1,"[1, 0, 0]",3
1,1.405099,3.010331,7.995463,3.127252,2.375624,0.132364,2.440177,1,"[1, 0, 0]",3
2,1.349386,2.911157,8.212341,2.979167,2.378475,0.350934,2.375677,1,"[1, 0, 0]",3
3,1.306893,2.880165,8.126134,2.997748,2.408411,0.293724,2.365830,1,"[1, 0, 0]",3
4,1.524079,3.097107,8.197822,3.185811,2.538845,0.176182,2.548006,1,"[1, 0, 0]",3
5,1.357885,2.935950,8.122505,3.032658,2.360656,0.320119,2.440177,1,"[1, 0, 0]",3
6,1.387158,2.993802,7.984574,3.132320,2.322880,0.466265,2.569670,1,"[1, 0, 0]",3
7,1.332389,2.913223,8.086207,3.051802,2.353528,0.351064,2.461841,1,"[1, 0, 0]",3
8,1.570349,3.194215,7.937387,3.408221,2.469708,0.265249,2.893648,1,"[1, 0, 0]",3
9,1.552408,3.150826,8.058076,3.313063,2.498218,0.256017,2.724274,1,"[1, 0, 0]",3
